In [53]:
import pandas as pd

tournaments = pd.read_csv('../Data Files/tournament.csv')
players = pd.read_csv('../Data Files/player.csv')
matches = pd.read_csv('../Data Files/match.csv')
profiles = pd.read_csv('../Data Files/playerProfile.csv')
country = pd.read_csv('../Data Files/country.csv')
arbiter = pd.read_csv('../Data Files/arbiter.csv')

In [54]:
from borb.pdf import Document
from borb.pdf.page.page import Page
from borb.pdf.pdf import PDF
from borb.pdf.canvas.layout.page_layout.multi_column_layout import SingleColumnLayout
from borb.pdf.canvas.layout.page_layout.page_layout import PageLayout
from decimal import Decimal
from borb.pdf.canvas.layout.image.image import Image

In [55]:
from borb.pdf import FixedColumnWidthTable, FlexibleColumnWidthTable
from borb.pdf.canvas.layout.text.paragraph import Paragraph
from pathlib import Path
from borb.pdf import HexColor
from borb.pdf.canvas.font.simple_font.true_type_font import TrueTypeFont
import requests
from borb.pdf.canvas.font.font import Font
from borb.pdf import Alignment
from borb.pdf import TableCell

In [56]:
font_path: Path = Path('../Fonts/Everson Mono.ttf')
custom_font: Font = TrueTypeFont.true_type_font_from_file(font_path)
imgAttrib = 'image: Flaticon.com'

In [57]:
pdf = Document()
page = Page()

pdf.add_page(page)

layout = SingleColumnLayout(page)

layout.add(    
    Image(
        Path('../strategy.png'),        
    width=Decimal(128),        
    height=Decimal(128), horizontal_alignment=Alignment.CENTERED)
    )

layout.add(Paragraph('Gambit Chess Tournament Report', font_size=22, font=custom_font, horizontal_alignment=Alignment.CENTERED))

In [58]:
playerID = 'BN-21-NJLH'

player = players[players['Player_ID'] == playerID]

playerName = player['Player_Name'].values[0]
dob = player['DOB'].values[0]
gender = player['Gender'].values[0]
fidePerc = player['FIDE_Percentile'].values[0]
rating = player['Player_Rating'].values[0]
countryName = country[country['Country_ID'] == player['Country'].values[0]]['Country_Name'].values[0]

In [59]:
layout.add(Paragraph(''))
layout.add(Paragraph(''))
layout.add(Paragraph(''))

layout.add(Paragraph('Player Name: ' + playerName, font_size=14, font=custom_font))
layout.add(Paragraph('License: ' + playerID, font_size=12, font=custom_font))
layout.add(Paragraph('Date of Birth: ' + dob, font_size=14, font=custom_font))
layout.add(Paragraph('Gender: ' + gender, font_size=14, font=custom_font))
layout.add(Paragraph('Rating: ' + str(players[players['Player_ID'] == playerID]['Player_Rating'].values[0]), font_size=12, font=custom_font))
layout.add(Paragraph('FIDE Percentile: ' + str(fidePerc), font_size=14, font=custom_font))
layout.add(Paragraph('Country: ' + countryName, font_size=14, font=custom_font))
layout.add(Paragraph(''))


In [60]:
page2 = Page()

pdf.add_page(page2)

layout2 = SingleColumnLayout(page2)

layout2.add(Paragraph('Achievements', font_size=12, font=custom_font))
prof = profiles[profiles['Player_ID'] == playerID]
prof = prof[prof['Achievements'].astype(str) != 'nan']

table = FixedColumnWidthTable(number_of_rows=len(prof) + 1, number_of_columns=3)

odd_color = HexColor("BBBBBB")
even_color = HexColor("FFFFFF")

table.add(TableCell(Paragraph('Achievement', font_size=12, font=custom_font), background_color=odd_color, padding_top=Decimal(5), padding_left=Decimal(5)))
table.add(TableCell(Paragraph('Year', font_size=12, font=custom_font), background_color=odd_color, padding_top=Decimal(5), padding_left=Decimal(5)))
table.add(TableCell(Paragraph('Coach', font_size=12, font=custom_font), background_color=odd_color, padding_top=Decimal(5), padding_left=Decimal(5)))
    
for i in range(len(prof)):
    c = even_color if i % 2 == 0 else odd_color
    row = prof.iloc[i]
    table.add(TableCell(Paragraph(row['Achievements'][:-3], font_size=10, font=custom_font), background_color=c, padding_top=Decimal(5), padding_left=Decimal(5)))
    table.add(TableCell(Paragraph(str(row['Year']), font_size=10, font=custom_font), background_color=c, padding_top=Decimal(5), padding_left=Decimal(5)))
    table.add(TableCell(Paragraph(row['CoachName'], font_size=10, font=custom_font), background_color=c, padding_top=Decimal(5), padding_left=Decimal(5)))

if (len(prof)>0):
    layout2.add(table)
else:
    layout2.add(Paragraph('None', font_size=12, font=custom_font))

In [61]:
with open('../Outs/player_test.pdf', 'wb') as pdfOut:
    PDF.dumps(pdfOut, pdf)